# Welcome to visualizing AWS GOES data!
#### **Audience:** Anybody with a computer and access to at least 4GB of memory.
#### **Intent:** Build familiarity with GOES-R data for use in scientific research.

This notebook is an introduction to accessing GOES-R (GOES-16, GOES-17, and GOES-18) satellite data from the AWS Cloud. No coding experience is required to complete this notebook, as all options are presented as Widgets, where you can select your desired product from a drop-down menu. As such, this notebook is meant for workshops, classrooms, or places where you don't want to spend time de-bugging code. If you would like to use this concept for analysis beyond an introduction, I recommend using aws_GOES_barebones.ipynb provided alongside this notebook. 

# Importing

In [ ]:
import ipywidgets as widgets
import requests
import netCDF4
import s3fs
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
import cartopy.crs as ccrs
from cartopy import feature as cf
import xarray
from datetime import datetime

import warnings
warnings.simplefilter('ignore')

# Defining

In [ ]:
def aws_satellite():
    
    global satellite_choice

    # Formatting settings for drop-down menus
    style = {'description_width':'120px'}
    layout = widgets.Layout(width='325px')

    # Display drop-down menus
    print("Select the satellite you'd like to use, then run the next cell to select a date.")
    satellite_choice = widgets.Dropdown(options=[('GOES-16 (East)', "noaa-goes16"), ("GOES-17 (Retired West)",'noaa-goes17'), ("GOES-18 (West)",'noaa-goes18')], description='GOES satellite:', style=style, layout=layout)

    return satellite_choice

In [ ]:
def aws_product(satellite):
    
    global product_choice, scan_choice
        
    aws = s3fs.S3FileSystem(anon=True)
        
    # Formatting settings for drop-down menus
    style = {'description_width':'120px'}
    layout = widgets.Layout(width='325px')

    # Display drop-down menus
    print("Select the satellite product you'd like to use, then run the next cell to select a date.")
    product_choice = widgets.Dropdown(options=[("Cloud Top Height", "ABI-L2-ACHA"), ("Cloud Top Temperature", "ABI-L2-ACHT"), ("Derived Motion Winds", "ABI-L2-DMW"),("Fire Hot Spot Characterization","ABI-L2-FDC"),("Land Surface Temperature", "ABI-L2-LST"),("Lightning Detection", "GLM-L2-LCFA"),("Rainfall Rate", "ABI-L2-RRQPEF"), ("Sea Surface Temperature","ABI-L2-SSTF"),("Smoke Detection", "ABI-L2-ADP"),("True Color Data", "ABI-L2-MCMIP")], description='GOES product:', style=style, layout=layout)
    scan_choice = widgets.Dropdown(options=[("Full Disk","F"),("CONUS/PACUS","C"),("Meso","M")], description='GOES scan:', style=style, layout=layout)
    
    display(product_choice, scan_choice)

In [ ]:
def aws_date(satellite, product, scan):
    
    global year, day, shour, smin, ehour, emin
        
    aws = s3fs.S3FileSystem(anon=True)
    
    # Search available year directories by product
    one_scan = ["GLM-L2-LCFA", "ABI-L2-RRQPEF", "ABI-L2-SSTF"] #products with only one scan
    
    if product not in one_scan:
        years = aws.ls(f'{satellite}/{product}{scan}/', refresh=True)  
    else: 
        years = aws.ls(f'{satellite}/{product}/', refresh=True)  
    
    year_options = []
    
    for y in years:
        year_options.append(y[-4:])
        
    # Formatting settings for drop-down menus
    style = {'description_width':'120px'}
    layout = widgets.Layout(width='325px')
    
    year_days = range(1,367,1)
    
    # Display drop-down menus
    year = widgets.Dropdown(options=[y for y in year_options], description='Year:', style=style, layout=layout)
    day = widgets.Dropdown(options=[d for d in year_days], description='Day of year:', style=style, layout=layout)
    shour = widgets.Dropdown(options=[('00'), ('01'), ('02'), ('03'), ('04'), ('05'), ('06'), ('07'), ('08'), ('09'), ('10'), ('11'), ('12'), ('13'), ('14'), ('15'), ('16'), ('17'), ('18'), ('19'), ('20'), ('21'), ('22'), ('23')], description='Start Hour:', style=style, layout=layout)
    smin = widgets.Dropdown(options=[('00'), ('01'), ('02'), ('03'), ('04'), ('05'), ('06'), ('07'), ('08'), ('09'), ('10'), ('11'), ('12'), ('13'), ('14'), ('15'), ('16'), ('17'), ('18'), ('19'), ('20'), ('21'), ('22'), ('23'), ('24'), ('25'), ('26'), ('27'), ('28'), ('29'), ('30'), ('31'), ('32'), ('33'), ('34'), ('35'), ('36'), ('37'), ('38'), ('39'), ('40'), ('41'), ('42'), ('43'), ('44'), ('45'), ('46'), ('47'), ('48'), ('49'), ('50'), ('51'), ('52'), ('53'), ('54'), ('55'), ('56'), ('57'), ('58'), ('59')], description='Start Minutes:', style=style, layout=layout)
    ehour = widgets.Dropdown(options=[('00'), ('01'), ('02'), ('03'), ('04'), ('05'), ('06'), ('07'), ('08'), ('09'), ('10'), ('11'), ('12'), ('13'), ('14'), ('15'), ('16'), ('17'), ('18'), ('19'), ('20'), ('21'), ('22'), ('23')], description='End Hour:', style=style, layout=layout)
    emin = widgets.Dropdown(options=[('00'), ('01'), ('02'), ('03'), ('04'), ('05'), ('06'), ('07'), ('08'), ('09'), ('10'), ('11'), ('12'), ('13'), ('14'), ('15'), ('16'), ('17'), ('18'), ('19'), ('20'), ('21'), ('22'), ('23'), ('24'), ('25'), ('26'), ('27'), ('28'), ('29'), ('30'), ('31'), ('32'), ('33'), ('34'), ('35'), ('36'), ('37'), ('38'), ('39'), ('40'), ('41'), ('42'), ('43'), ('44'), ('45'), ('46'), ('47'), ('48'), ('49'), ('50'), ('51'), ('52'), ('53'), ('54'), ('55'), ('56'), ('57'), ('58'), ('59')], description='End Minutes:', style=style, layout=layout)

    start_time = widgets.HBox([shour, smin])
    end_time = widgets.HBox([ehour, emin])

    print("Select your chosen date and time, chosen from the orbit track, then run the next cell.")
    display(year, day)
    display(start_time, end_time)

In [ ]:
def aws_data(satellite, product, scan, year, day, shour, ehour, smin, emin):

    global fnames
    
    fnames = []
    
    if len(str(day)) == 1:
        doy = (f"00{day}")
    elif len(str(day)) == 2:
        doy = (f"0{day}")
    elif len(str(day)) == 3:
        doy = (f"{day}")
            
    aws = s3fs.S3FileSystem(anon=True)
    
    # Search all requested hours for data files
    for hour in range(int(shour),int(ehour)+1):
        if len(str(hour)) == 1:
            hour = (f"0{hour}")
        if len(str(hour)) == 2:
            hour = (f"{hour}")
        
        one_scan = ["GLM-L2-LCFA", "ABI-L2-RRQPEF", "ABI-L2-SSTF"] #products with only one scan

        if product not in one_scan:
            data_files = aws.ls(f'{satellite}/{product}{scan}/{year}/{doy}/{hour}/', refresh=True)  
        else:
            data_files = aws.ls(f'{satellite}/{product}/{year}/{doy}/{hour}/', refresh=True)  

        # Extracting files for chosen time
        for file in data_files:
            file_start = file.split('_')[-3][8:12]
            file_end = file.split('_')[-2][8:12]
            if ((int(file_start[0:4]) >= int((str(shour) + str(smin)))) and (int(file_end[0:4]) <= int((str(ehour) + str(emin))))):
                fnames.append(file)
                
    print(f"Retrieved {len(fnames)} files from your criteria.")

In [ ]:
def aws_plot(satellite, product):        
            #for f in range(0, len(fnames)):  # Uncomment to plot all images
            f = 0
        
            fname = (fnames[f])[12:]

            # Load dataset
            resp = requests.get(f'https://{satellite}.s3.amazonaws.com/{fname}')
            nc = netCDF4.Dataset(fname, memory = resp.content)

            dataset = xarray.open_dataset(xarray.backends.NetCDF4DataStore(nc))
            dataset.load()
                
            # Aerosol Detection! 
            if product == "ABI-L2-ADP":
                dat = dataset.metpy.parse_cf('Smoke')
                
                # Three variables to choose from
                smo = dataset['Smoke'].data
                dus = dataset['Dust'].data
                aer = dataset['Aerosol'].data
                
                # Identify plotting variable 
                plotting_var = smo
                
                # Identify plotting coordinates
                x = dat.x
                y = dat.y
                geos = dat.metpy.cartopy_crs
                
                # Identify mask options
                label_list = ['No smoke','Smoke']
            
                # Plotting
                fig = plt.figure(figsize=(12, 7), dpi=100)
                ax = plt.axes(projection=geos)
                cmap = plt.get_cmap('Set3', 2)
                plt.imshow(plotting_var, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=geos, cmap = cmap)
                
                cbar = plt.colorbar(cmap = cmap)
                cbar.set_ticks([])
                for j, lab in enumerate(label_list):
                    cbar.ax.text(1.5, (2 * j + 1) / (len(label_list)*2), lab, ha='left', va='baseline')   
                                       
                title = "Smoke Detection"
            
            # Cloud Top Height!
            elif product == "ABI-L2-ACHA":
                dat = dataset.metpy.parse_cf('HT')
                
                # Identify variable 
                plotting_var = dataset['HT'].data

                # Identify plotting coordinates
                x = dat.x
                y = dat.y
                geos = dat.metpy.cartopy_crs
                
                # Plotting
                fig = plt.figure(figsize=(12, 7), dpi=100)
                ax = plt.axes(projection=geos)
                plt.imshow(plotting_var, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=geos)   
                
                units = dataset['HT'].units
                plt.colorbar(cmap = "jet", label = units)
                
                title = "Cloud Top Height"
                
            # Cloud Top Temperature!
            elif product == "ABI-L2-ACHT":
                dat = dataset.metpy.parse_cf('TEMP')
                
                # Identify variable and transform to Celsius
                plotting_var = dataset['TEMP'].data - 273.15

                # Identify plotting coordinates
                x = dat.x
                y = dat.y
                geos = dat.metpy.cartopy_crs
                
                # Plotting
                fig = plt.figure(figsize=(12, 7), dpi=100)
                ax = plt.axes(projection=geos)
                plt.imshow(plotting_var, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=geos)   
                
                units = "Celsius"
                plt.colorbar(cmap = "jet", label = units)
                
                title = "Cloud Top \nTemperature"

            # Derived Motion Winds!
            elif product == "ABI-L2-DMW":      
                dat = dataset.metpy.parse_cf("wind_speed")

                # Isolating 850 mb winds
                mb850 = dataset.where(dataset.pressure >= 800, drop = False)
                mb850 = mb850.where(mb850.pressure <= 900, drop = False)
                speed850 = mb850['wind_speed'].data
                direction850 = mb850['wind_direction'].data
                u850= -1*speed850*np.sin((np.pi/180)*direction850)
                v850= -1*speed850*np.cos((np.pi/180)*direction850) 
                
                # Isolating 300 mb winds
                mb300 = dataset.where(dataset.pressure >= 250, drop = False)
                mb300 = mb300.where(mb300.pressure <= 350, drop = False)
                speed300 = mb300['wind_speed'].data
                direction300 = mb300['wind_direction'].data
                u300= -1*speed300*np.sin((np.pi/180)*direction300)
                v300= -1*speed300*np.cos((np.pi/180)*direction300) 
                
                # Isolating 500 mb winds
                mb500 = dataset.where(dataset.pressure >= 450, drop = False)
                mb500 = mb500.where(mb500.pressure <= 550, drop = False)
                speed500 = mb500['wind_speed'].data
                direction500 = mb500['wind_direction'].data
                u500= -1*speed500*np.sin((np.pi/180)*direction500)
                v500= -1*speed500*np.cos((np.pi/180)*direction500) 
                
                # Identify plotting coordinates
                x = dat.lon
                y = dat.lat
                geos = dat.metpy.cartopy_crs  
                
                # Plotting
                fig = plt.figure(figsize=(12, 7), dpi=100)
                ax = plt.axes(projection=geos)
                ax.set_extent([x.min(),x.max(),y.min(),y.max()], geos)
                ax.barbs(x, y, u850, v850, length = 6, color = "red", label = "850 mb")
                ax.barbs(x, y, u500, v500, length = 6, color = "orange", label = "500 mb")
                ax.barbs(x, y, u300, v300, length = 6, color = "green", label = "300 mb")
                
                ax.legend()
                
                title = "Derived Motion Winds"
                
            # Fire Hot Spot! 
            elif product == "ABI-L2-FDC":
                dat = dataset.metpy.parse_cf('DQF')  

                # Identify variable
                plotting_var = dataset['DQF'].data 
    
                # Identify plotting coordinates
                x = dat.x
                y = dat.y
                geos = dat.metpy.cartopy_crs
                
                # Plotting
                fig = plt.figure(figsize=(12, 7), dpi=100)
                ax = plt.axes(projection=geos)
                cmap = plt.get_cmap('Set3', 6)
                plt.imshow(plotting_var, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=geos,cmap=cmap)   

                label_list = ['Fire','No Fire','Opaque Cloud','Invalid Pixel','Invalid Pixel','Invalid Pixel']
                cbar = plt.colorbar(cmap = cmap)
                cbar.set_ticks([])
                for j, lab in enumerate(label_list):
                    cbar.ax.text(1.5, (2 * j + 1) / (len(label_list)*2), lab, ha='left', va='baseline')   
                    
                #units = dataset['Mask'].units
                #color_options = ['FFFF00','8BC34A','B3E5FC','2196F3', 'B2EBF2','C5CAE9']
                #plt.colorbar(cmap = "jet")
                title = "Fire Hot Spot"
                
            # Land Surface Temperature!
            elif product == "ABI-L2-LST":
                dat = dataset.metpy.parse_cf('LST')
                
                # Identify variable and transform to Celsius
                plotting_var = dataset['LST'].data - 273.15
                
                # Identify plotting coordinates
                x = dat.x
                y = dat.y
                geos = dat.metpy.cartopy_crs
                
                # Plotting
                fig = plt.figure(figsize=(12, 7), dpi=100)
                ax = plt.axes(projection=geos)
                plt.imshow(plotting_var, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=geos)   
                
                units = "Celsius"
                plt.colorbar(cmap = "jet", label = units)
                
                title = "Land Surface Temperature"
 
            # Lightning Detection!
            elif product == "GLM-L2-LCFA":
                dat = dataset.metpy.parse_cf('flash_energy')
                
                # Identify variable
                flash_energy = dataset['flash_energy'].data 
                
                # Identify plotting coordinates
                x_flash = dat.flash_lat
                y_flash = dat.flash_lon
                geos = dat.metpy.cartopy_crs  
                
                # Plotting
                fig = plt.figure(figsize=(12, 7), dpi=100)
                ax = plt.axes(projection=geos)
                lightning = plt.scatter(y_flash, x_flash,c = flash_energy, s = 5, label = "Flash Energy")
                
                units = dataset['flash_energy'].units 
                plt.colorbar(lightning, label = units)
                
                title = "Lightning Detection"
                
            # Rainfall Rate!
            elif product == "ABI-L2-RRQPEF":
                dat = dataset.metpy.parse_cf('RRQPE')
                
                # Identify variable
                plotting_var = dataset['RRQPE'].data 

                # Identify plotting coordinates
                x = dat.x
                y = dat.y
                geos = dat.metpy.cartopy_crs
                
                # Plotting
                fig = plt.figure(figsize=(12, 7), dpi=100)
                ax = plt.axes(projection=geos)
                plt.imshow(plotting_var, origin='upper',vmin=0, vmax=20, extent=(x.min(), x.max(), y.min(), y.max()), transform=geos)  
                
                units = dataset['RRQPE'].units
                plt.colorbar(cmap = "jet", label = units)
                
                title = "Rainfall Rate: \nQuantitative Precipitation \nEstimate"
    
            # Sea Surface Temperature!
            elif product == "ABI-L2-SSTF":
                dat = dataset.metpy.parse_cf('SST')
                
                # Only pixels with DQF = 0 should be used
                qc_dataset = dataset.where(dataset.DQF == 0, drop = False)    
                
                # Identify variable and transform to Celsius
                plotting_var = qc_dataset['SST'].data - 273.15
                                
                # Identify plotting coordinates
                x = dat.x
                y = dat.y
                geos = dat.metpy.cartopy_crs
                
                # Plotting
                fig = plt.figure(figsize=(12, 7), dpi=100)
                ax = plt.axes(projection=geos)
                plt.imshow(plotting_var, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=geos)   
                
                units = "Celsius"
                plt.colorbar(cmap = "jet", label = units)
                
                title = "Sea Surface (Skin) \nTemperature"
                
            # True color data!
            elif product == "ABI-L2-MCMIP": 
                dat = dataset.metpy.parse_cf('CMI_C02')
                gamma = 2.2
                r = dataset['CMI_C02'].data; r = np.clip(r, 0, 1); r = np.power(r, 1/gamma)
                g = dataset['CMI_C03'].data; g = np.clip(g, 0, 1); g = np.power(g, 1/gamma)
                b = dataset['CMI_C01'].data; b = np.clip(b, 0, 1); b = np.power(b, 1/gamma)

                # Calculate the "True" Green
                g_true = 0.45 * r + 0.1 * g + 0.45 * b
                g_true = np.clip(g_true, 0, 1) 
                
                # Create a single RGB image for plotting
                plotting_var = np.dstack((r, g_true, b))
                
                # Identify plotting coordinates
                x = dat.x
                y = dat.y
                geos = dat.metpy.cartopy_crs
                
                # Plotting
                fig = plt.figure(figsize=(12, 7), dpi=100)
                ax = plt.axes(projection=geos)
                plt.imshow(plotting_var, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=geos)  
                
                title = "True Color Data"
                  
            nc.close()

            # For all plots
            ax.coastlines(resolution='50m', color='black', linewidth=0.25)
            ax.add_feature(ccrs.cartopy.feature.STATES, linewidth=0.25)
    
            scan_start = datetime.strptime(dataset.time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')
            
            plt.title(f"GOES-{satellite[-2:]} {title}", loc='left', fontweight='bold')
            plt.title('{}'.format(scan_start.strftime('%d %B %Y at %H:%M UTC')), loc='right')

            plt.show()
            plt.close(fig)
            plt.close("all")

In [ ]:
def aws_animate(satellite, product, scan):
    
    # Lightning Detection!
    if product == "GLM-L2-LCFA":
        
        # Reading in true color data
        fnames_truecolor = []

        day_str = str(day.value)
        if len(day_str) == 1:
            doy = (f"00{day_str}")
        elif len(day_str) == 2:
            doy = (f"0{day_str}")
        elif len(day_str) == 3:
            doy = (f"{day_str}")

        aws = s3fs.S3FileSystem(anon=True)

        for hour in range(int(shour.value),int(ehour.value)+1):
                if len(str(hour)) == 1:
                    hour = (f"0{hour}")
                if len(str(hour)) == 2:
                    hour = (f"{hour}")

        data_files = aws.ls(f'{satellite}/{"ABI-L2-MCMIP"}{scan}/{year.value}/{doy}/{hour}/', refresh=True)  

        for file in data_files:
                file_start = file.split('_')[-3][8:12]
                file_end = file.split('_')[-2][8:12]
                if ((int(file_start[0:4]) >= int((str(shour.value) + str(smin.value)))) and (int(file_end[0:4]) <= int((str(ehour.value) + str(emin.value))))):
                    fnames_truecolor.append(file)
        fig, ax = plt.subplots()

        def animate(i):    
            if len(fnames_truecolor) > 1:
                fname_truecolor = (fnames_truecolor[(round((len(fnames_truecolor)/len(fnames))*i))])[12:]    
            else:
                fname_truecolor = (fnames_truecolor[0])[12:]
            
            resp_truecolor = requests.get(f'https://{satellite}.s3.amazonaws.com/{fname_truecolor}')
            nc_truecolor = netCDF4.Dataset(fname_truecolor, memory = resp_truecolor.content)
            dataset_truecolor = xarray.open_dataset(xarray.backends.NetCDF4DataStore(nc_truecolor))
            dataset_truecolor.load()
            dat_truecolor = dataset_truecolor.metpy.parse_cf('CMI_C02')
            gamma = 2.2
            r = dataset_truecolor['CMI_C02'].data; r = np.clip(r, 0, 1); r = np.power(r, 1/gamma)
            g = dataset_truecolor['CMI_C03'].data; g = np.clip(g, 0, 1); g = np.power(g, 1/gamma)
            b = dataset_truecolor['CMI_C01'].data; b = np.clip(b, 0, 1); b = np.power(b, 1/gamma)

            # Calculate the "True" Green
            g_true = 0.45 * r + 0.1 * g + 0.45 * b
            g_true = np.clip(g_true, 0, 1) 
            plotting_var = np.dstack((r, g_true, b))

            # Identify plotting coordinates
            x_truecolor = dat_truecolor.x
            y_truecolor = dat_truecolor.y

            geos_truecolor = dat_truecolor.metpy.cartopy_crs

            fig.clear()
            ax = fig.add_subplot(projection=geos_truecolor)

            #lightning data 
            fname = (fnames[i])[12:]
            resp = requests.get(f'https://{satellite}.s3.amazonaws.com/{fname}')
            nc = netCDF4.Dataset(fname, memory = resp.content)
            dataset = xarray.open_dataset(xarray.backends.NetCDF4DataStore(nc))
            dataset.load()
            dat = dataset.metpy.parse_cf('flash_energy')
            x = dat.flash_lat
            y = dat.flash_lon
            geos = dat.metpy.cartopy_crs 
            
            dataset.close()
            dataset_truecolor.close()

            plt.imshow(plotting_var, origin='upper', extent=(x_truecolor.min(), x_truecolor.max(), y_truecolor.min(), y_truecolor.max()), transform=geos_truecolor)
            ax.scatter(y, x, c = "blue", s=5, transform=geos)

            ax.coastlines(resolution='50m', color='black', linewidth=0.25)
            ax.add_feature(ccrs.cartopy.feature.STATES, linewidth=0.25)
            ax.set_extent([x_truecolor.min(), x_truecolor.max(), y_truecolor.min(), y_truecolor.max()], geos_truecolor)
            
            title = "Lightning Detection"
            scan_start = datetime.strptime(dataset.time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')
            plt.title(f"GOES-{satellite[-2:]} {title} \n{scan_start.strftime('%d %B %Y at %H:%M UTC')}", loc='left', fontweight='bold')

        ani = animation.FuncAnimation(fig, animate, repeat=True, frames=len(fnames)-1, interval=200)
        ani.save('GOES_animation.gif', writer='pillow')
        plt.close("all")
        
    # Land Surface Temperature!
    elif product == "ABI-L2-LST":
        
        fig, ax = plt.subplots()
        
        def animate(i):    
            fname = (fnames[i])[12:]
            resp = requests.get(f'https://{satellite}.s3.amazonaws.com/{fname}')
            nc = netCDF4.Dataset(fname, memory = resp.content)
            dataset = xarray.open_dataset(xarray.backends.NetCDF4DataStore(nc))
            dataset.load()
            dat = dataset.metpy.parse_cf('LST')
                
            # Identify variable and transform to Celsius
            plotting_var = dataset['LST'].data - 273.15
            # Identify plotting coordinates
            x = dat.x
            y = dat.y

            geos = dat.metpy.cartopy_crs
            
            dataset.close()

            fig.clear()
            ax = fig.add_subplot(projection=geos)
            plt.imshow(plotting_var, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=geos, vmin=-40, vmax=40)

            ax.coastlines(resolution='50m', color='black', linewidth=0.25)
            ax.add_feature(ccrs.cartopy.feature.STATES, linewidth=0.25)
            ax.set_extent([x.min(), x.max(), y.min(), y.max()], geos)
            title = "Land Surface Temperature"
            scan_start = datetime.strptime(dataset.time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')
            plt.title(f"GOES-{satellite[-2:]} {title} \n{scan_start.strftime('%d %B %Y at %H:%M UTC')}", loc='left', fontweight='bold')
            plt.colorbar(label = "Celsius")
        ani = animation.FuncAnimation(fig, animate, repeat=True, frames=len(fnames)-1, interval=200)
        ani.save('GOES_LST_animation.gif', writer='pillow')
        plt.close("all")
        plt.close(fig)
        
    # Cloud Top Temperature!
    elif product == "ABI-L2-ACHT":
        
        fig, ax = plt.subplots()
        
        def animate(i):    
            fname = (fnames[i])[12:]
            resp = requests.get(f'https://{satellite}.s3.amazonaws.com/{fname}')
            nc = netCDF4.Dataset(fname, memory = resp.content)
            dataset = xarray.open_dataset(xarray.backends.NetCDF4DataStore(nc))
            dataset.load()
            dat = dataset.metpy.parse_cf('TEMP')
                
            # Identify variable and transform to Celsius
            plotting_var = dataset['TEMP'].data - 273.15
            # Identify plotting coordinates
            x = dat.x
            y = dat.y

            geos = dat.metpy.cartopy_crs

            dataset.close()
            
            fig.clear()
            ax = fig.add_subplot(projection=geos)

            plt.imshow(plotting_var, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=geos, vmin=-80, vmax=20)

            ax.coastlines(resolution='50m', color='black', linewidth=0.25)
            ax.add_feature(ccrs.cartopy.feature.STATES, linewidth=0.25)
            ax.set_extent([x.min(), x.max(), y.min(), y.max()], geos)
            title = "Cloud Top Temperature"
            scan_start = datetime.strptime(dataset.time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')
            plt.title(f"GOES-{satellite[-2:]} {title} \n{scan_start.strftime('%d %B %Y at %H:%M UTC')}", loc='left', fontweight='bold')
            plt.colorbar(label = "Celsius")
            
        ani = animation.FuncAnimation(fig, animate, repeat=True, frames=len(fnames)-1, interval=200)
        ani.save('GOES_CTT_animation.gif', writer='pillow')
        plt.close("all")
        plt.close(fig)
            
    else:
        print("oops, i can't animate that yet")

# Choosing your variables!

In [ ]:
aws_satellite()

In [ ]:
aws_product(satellite_choice.value)

In [ ]:
aws_date(satellite_choice.value, product_choice.value, scan_choice.value)

In [ ]:
aws_data(satellite_choice.value, product_choice.value, scan_choice.value, year.value, day.value, shour.value, ehour.value, smin.value, emin.value)

In [ ]:
aws_animate(satellite_choice.value, product_choice.value, scan_choice.value)

In [ ]:
aws_plot(satellite_choice.value, product_choice.value)

In [ ]:
# Your analysis here!

**GOES Product Documentation:**
- https://www.goes-r.gov/products/ (Broad explanation of all products)
- https://www.star.nesdis.noaa.gov/goesr/ (Algorithm & product description)
- http://cimss.ssec.wisc.edu/goes/OCLOFactSheetPDFs/ (Quick guides)
- https://github.com/awslabs/open-data-docs/blob/main/docs/noaa/noaa-goes16/README.md (Variables & product names) 

**Coding Sources:**
- https://unidata.github.io/python-gallery/examples/mapping_GOES16_TrueColor.html#sphx-glr-download-examples-mapping-goes16truecolor-py   
- https://unidata.github.io/python-training/gallery/mapping_goes16_truecolor/   
- https://fire.trainhub.eumetsat.int/docs/figure3_GOES17_L1B_and_L2.html   
- https://unidata.github.io/python-training/gallery/mapping_goes16_truecolor/

**CSP Access:**    
AWS: https://registry.opendata.aws/noaa-goes/   
Google: https://console.cloud.google.com/marketplace/product/noaa-public/goes   
Azure: https://microsoft.github.io/AIforEarthDataSets/data/goes-r.html   